# Генеративне мреже

Рекурентне неуронске мреже (RNNs) и њихове варијанте са контролисаним ћелијама, као што су Long Short Term Memory ћелије (LSTMs) и Gated Recurrent Units (GRUs), пружају механизам за моделирање језика, односно могу да науче редослед речи и дају предвиђања за следећу реч у низу. Ово нам омогућава да користимо RNNs за **генеративне задатке**, као што су обично генерисање текста, машински превод, па чак и генерисање описа слика.

У RNN архитектури коју смо разматрали у претходној јединици, свака RNN јединица је производила следеће скривено стање као излаз. Међутим, можемо додати још један излаз свакој рекурентној јединици, што би нам омогућило да добијемо **низ** (који је једнак по дужини оригиналном низу). Штавише, можемо користити RNN јединице које не прихватају улаз на сваком кораку, већ само узимају неки почетни вектор стања и затим производе низ излаза.

У овом нотебуку, фокусираћемо се на једноставне генеративне моделе који нам помажу да генеришемо текст. Ради једноставности, направимо **мрежу на нивоу карактера**, која генерише текст слово по слово. Током тренинга, потребно је узети неки корпус текста и поделити га на низове слова.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## Изградња речника карактера

Да бисмо изградили генеративну мрежу на нивоу карактера, потребно је да текст поделимо на појединачне карактере уместо на речи. `TextVectorization` слој који смо раније користили не може то да уради, па имамо две опције:

* Ручно учитавање текста и ручна токенизација, као у [овом званичном примеру из Keras-а](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* Коришћење `Tokenizer` класе за токенизацију на нивоу карактера.

Одлучићемо се за другу опцију. `Tokenizer` се такође може користити за токенизацију на нивоу речи, па би требало да буде лако прећи са токенизације на нивоу карактера на токенизацију на нивоу речи.

Да бисмо извршили токенизацију на нивоу карактера, потребно је да проследимо параметар `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

Такође желимо да користимо један посебан токен за означавање **краја секвенце**, који ћемо назвати `<eos>`. Хајде да га ручно додамо у вокабулар:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

Сада, да бисмо кодирали текст у низове бројева, можемо користити:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## Тренирање генеративне RNN за генерисање наслова

Начин на који ћемо тренирати RNN да генерише наслове вести је следећи. У сваком кораку, узимамо један наслов, који ће бити унет у RNN, и за сваки улазни карактер тражимо од мреже да генерише следећи излазни карактер:

![Слика која приказује пример генерације речи 'HELLO' помоћу RNN.](../../../../../translated_images/sr/rnn-generate.56c54afb52f9781d.webp)

За последњи карактер у нашој секвенци, тражићемо од мреже да генерише `<eos>` токен.

Главна разлика између генеративне RNN коју овде користимо је у томе што ћемо узимати излаз из сваког корака RNN-а, а не само из последње ћелије. Ово се може постићи постављањем параметра `return_sequences` за RNN ћелију.

Дакле, током тренирања, улаз у мрежу биће секвенца кодираних карактера одређене дужине, а излаз ће бити секвенца исте дужине, али померена за један елемент и завршена са `<eos>`. Минибатч ће се састојати од неколико таквих секвенци, и биће потребно користити **попуњавање** (padding) да би се све секвенце ускладиле.

Хајде да направимо функције које ће трансформисати скуп података за нас. Пошто желимо да попуњавамо секвенце на нивоу минибатча, прво ћемо груписати скуп података позивом `.batch()`, а затим га `map`-овати како бисмо извршили трансформацију. Дакле, функција за трансформацију ће узимати читав минибатч као параметар:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

Неколико важних ствари које овде радимо:
* Прво извлачимо стварни текст из стринг тензора
* `text_to_sequences` претвара листу стрингова у листу целобројних тензора
* `pad_sequences` допуњава те тензоре до њихове максималне дужине
* На крају вршимо one-hot енкодирање свих карактера, као и померање и додавање `<eos>`. Ускоро ћемо видети зашто су нам потребни one-hot енкодирани карактери

Међутим, ова функција је **Pythonic**, тј. не може се аутоматски превести у Tensorflow рачунски граф. Добићемо грешке ако покушамо да користимо ову функцију директно у `Dataset.map` функцији. Потребно је да ову Pythonic функцију обухватимо коришћењем `py_function` омотача:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **Note**: Разликовање између Pythonic и Tensorflow функција трансформације може изгледати превише сложено, и можда се питате зашто не трансформишемо скуп података користећи стандардне Python функције пре него што га проследимо функцији `fit`. Иако је то дефинитивно могуће, коришћење `Dataset.map` има огромну предност, јер се процес трансформације података извршава користећи Tensorflow рачунски граф, који користи предности GPU обраде и минимизује потребу за преносом података између CPU/GPU.

Сада можемо изградити нашу генераторску мрежу и започети тренинг. Она може бити заснована на било којој рекурентној ћелији коју смо разматрали у претходној јединици (једноставна, LSTM или GRU). У нашем примеру користићемо LSTM.

Пошто мрежа узима карактере као улаз, а величина вокабулара је прилично мала, не треба нам слој за уграђивање (embedding layer); улаз кодирани у one-hot формату може директно ући у LSTM ћелију. Излазни слој биће `Dense` класификатор који ће претворити излаз LSTM-а у бројеве токена кодиране у one-hot формату.

Поред тога, пошто радимо са секвенцама променљиве дужине, можемо користити слој `Masking` да креирамо маску која ће игнорисати попуњени (padded) део низа. Ово није строго неопходно, јер нас не занима превише све што иде изван `<eos>` токена, али ћемо га користити ради стицања искуства са овим типом слоја. `input_shape` биће `(None, vocab_size)`, где `None` означава секвенцу променљиве дужине, а излазни облик је такође `(None, vocab_size)`, као што можете видети из `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## Генерисање резултата

Сада када смо обучили модел, желимо да га користимо за генерисање резултата. Пре свега, потребан нам је начин за декодирање текста представљеног низом бројева токена. За то бисмо могли да користимо функцију `tokenizer.sequences_to_texts`; међутим, она не ради добро са токенизацијом на нивоу карактера. Због тога ћемо узети речник токена из токенизатора (назван `word_index`), направити обрнуту мапу и написати сопствену функцију за декодирање:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

Сада, хајде да започнемо са генерисањем. Почећемо са неким низом `start`, кодирати га у секвенцу `inp`, а затим ћемо на сваком кораку позвати нашу мрежу да предвиди следећи карактер.

Излаз мреже `out` је вектор са `vocab_size` елемената који представљају вероватноће за сваки токен, а највероватнији број токена можемо пронаћи коришћењем `argmax`. Затим додајемо овај карактер у генерисану листу токена и настављамо са генерисањем. Овај процес генерисања једног карактера понавља се `size` пута како бисмо генерисали потребан број карактера, а прекидамо раније ако се појави `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## Узимање узорка резултата током обуке

Пошто немамо корисне метрике као што је *тачност*, једини начин да видимо да ли наш модел постаје бољи је **узимање узорка** генерисаног низа током обуке. Да бисмо то урадили, користићемо **повратне позиве**, односно функције које можемо проследити функцији `fit`, а које ће бити позиване периодично током обуке.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


Овај пример већ генерише прилично добар текст, али може се додатно побољшати на неколико начина:
* **Више текста**. Користили смо само наслове за наш задатак, али можда желите да експериментишете са пуним текстом. Запамтите да RNN-ови нису баш добри у обради дугих секвенци, па има смисла или их поделити на краће реченице, или увек тренирати на фиксној дужини секвенце од неке унапред дефинисане вредности `num_chars` (рецимо, 256). Можете покушати да промените горњи пример у такву архитектуру, користећи [званични Керас туторијал](https://keras.io/examples/generative/lstm_character_level_text_generation/) као инспирацију.
* **Вишеслојни LSTM**. Има смисла пробати 2 или 3 слоја LSTM ћелија. Као што смо поменули у претходној јединици, сваки слој LSTM-а извлачи одређене обрасце из текста, а у случају генератора на нивоу карактера можемо очекивати да ће нижи LSTM слој бити одговоран за извлачење слогова, а виши слојеви - за речи и комбинације речи. Ово се може једноставно имплементирати прослеђивањем параметра броја слојева конструктору LSTM-а.
* Можда ћете желети да експериментишете и са **GRU јединицама** и видите које боље функционишу, као и са **различитим величинама скривених слојева**. Превелики скривени слој може довести до пренатренираности (нпр. мрежа ће научити тачан текст), а мања величина можда неће дати добар резултат.


## Генерисање меког текста и температура

У претходној дефиницији функције `generate`, увек смо узимали карактер са највећом вероватноћом као следећи карактер у генерисаном тексту. Ово је често доводило до тога да се текст "врти" између истих секвенци карактера изнова и изнова, као у овом примеру:
```
today of the second the company and a second the company ...
```

Међутим, ако погледамо расподелу вероватноћа за следећи карактер, може се десити да разлика између неколико највећих вероватноћа није велика, на пример, један карактер може имати вероватноћу 0.2, а други 0.19, итд. На пример, када тражимо следећи карактер у секвенци '*play*', следећи карактер може једнако добро бити размак или **е** (као у речи *player*).

Ово нас доводи до закључка да није увек "праведно" изабрати карактер са највећом вероватноћом, јер избор другог највећег може и даље довести до смисленог текста. Паметније је **узорковати** карактере из расподеле вероватноћа коју даје излаз мреже.

Ово узорковање може се обавити помоћу функције `np.multinomial`, која имплементира такозвану **мултиномијалну расподелу**. Функција која имплементира ово **меко** генерисање текста дефинисана је испод:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

Увели смо још један параметар који се зове **температура**, који се користи да означи колико строго треба да се држимо највеће вероватноће. Ако је температура 1.0, радимо правично мултиномијално узорковање, а када температура иде ка бесконачности - све вероватноће постају једнаке, и насумично бирамо следећи карактер. У примеру испод можемо приметити да текст постаје бесмислен када превише повећамо температуру, и подсећа на "циклично" тешко генерисан текст када се приближи 0.



---

**Одрицање од одговорности**:  
Овај документ је преведен коришћењем услуге за превођење помоћу вештачке интелигенције [Co-op Translator](https://github.com/Azure/co-op-translator). Иако се трудимо да превод буде тачан, молимо вас да имате у виду да аутоматски преводи могу садржати грешке или нетачности. Оригинални документ на његовом изворном језику треба сматрати ауторитативним извором. За критичне информације препоручује се професионални превод од стране људи. Не преузимамо одговорност за било каква погрешна тумачења или неспоразуме који могу настати услед коришћења овог превода.
